## **Tidying**

Part of the data engineering process is data cleaning and tidying. What is done in those two processes, is trying to make the data more readable, and complete. This makes much easier to analyze, visualize, and train the data.


## **Data Tidying**

Making the data more organized, and readable is the result of applying data tidying. 

In this section two main pandas functions are used in data tidying those are `melt` and `pivot_table`.

---

Let's start by taking a look at the below dataframe, which represents the income ranges based on religion. This is part of the PEW research, which is famous in the US for conducting pollings and surveys on citizens.

When the following are satisfied:
1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

We can then say that our dataset is *tidy*.

First we need to import pandas to read csv datasets.

# Importing  essential libraries


In [ ]:
import pandas as pd
import numpy as np

### **PEW Research Dataset**

### 1. Start by Importing the dataset into a pandas dataframe.

In [ ]:
file_path = './data/pew-raw.csv'
pew_data = pd.read_csv(file_path)

We load the dataset using a relative path and store it in a pandas DataFrame.A relative path is used for portability



###  2. Observe the dataset using the `loc`, `iloc`, `head`, or `tail` approaches

In [ ]:
# Define a helper class for inspecting datasets
class DataInspector:
    def __init__(self, dataframe):
        self.df = dataframe

    def show_basic_info(self):
        print("=== First 5 Rows ===")
        display(self.df.head())

        print("\n=== Last 5 Rows ===")
        display(self.df.tail())

        print("\n=== 3rd Row (using iloc) ===")
        display(self.df.iloc[2])

        print("\n=== Value in 'religion' Column of First Row ===")
        print(self.df.loc[0, 'religion'])

# Create an instance for the PEW dataset
pew_inspector = DataInspector(pew_data)

# Call the method to inspect
pew_inspector.show_basic_info()


> ###  Code Overview  
i used a single code block with a helper class to display key parts of the dataset cleanly:
- First and last few rows
- A specific row by index
- A selected cell by label

This avoids using multiple separate cells or `print()` statements, keeping the notebook output well-organized and professional.

---

***What does not seem right in the above dataframe?***

In the current dataset, income ranges (`<$10k`, `$10-20k`, etc.) are used as column headers. These are **values**, not variables. This violates tidy data principles:

- Column names should represent variables.
- Values should exist as rows under those variables.

To fix this, we use the `melt()` function to convert the income brackets into a single column, with corresponding frequencies in another column.


### 3. Try to make the column headers represent a variable not a value. For that, use the `melt` function.

In [ ]:
# Reshape the dataset using melt to follow tidy data principles
pew_tidy = pd.melt(
    pew_data,
    id_vars='religion',           # Keep 'religion' column fixed
    var_name='income_bracket',    # New column for income levels
    value_name='count'            # New column for the corresponding values
)

# Display the first few rows of the reshaped DataFrame
pew_tidy.head()


Reshape the Dataset Using `melt()`

We use the `melt()` function to convert income brackets from column headers into row values. This creates two new columns: one for `income_bracket` and one for the corresponding `count`. This follows tidy data principles and simplifies analysis.


> ---

## **Billboard Dataset**

This dataset outlines data about the top hit songs on the Billboard list and the week from entrance that it was in the billboard with the ranking.

### 1. Read the dataset and store it in a pandas dataframe. Note that the usual utf-8 encoding does not work on this dataset. The reason behind this is that there might be characters that are not supported by `utf-8`.

The suggestion is to use for this dataset `unicode_escape` encoding. (converts all non-ASCII characters into their \uXXXX representations)

In [1]:
import pandas as pd

# Step 1: Load the dataset using unicode_escape encoding
billboard_data = pd.read_csv('./data/billboard.csv', encoding='unicode_escape')


We load the dataset using `unicode_escape` encoding to handle any non-standard characters that would otherwise cause errors with `utf-8`.

The dataset contains weekly Billboard rankings of popular songs over time.


### 2. Observe the first few rows of the dataset.

In [2]:
# Display the first few rows
billboard_data.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We inspect the top rows to understand the structure and format of the dataset before applying any transformations. This helps identify issues such as non-tidy formatting, inconsistent column names, or missing values.


***What is wrong with the above dataset?***

The dataset is in a wide format where each column after the initial metadata represents a separate week. This violates tidy data principles:

- Week numbers are used as column headers instead of values in a single `week` column.
- Rankings are spread across multiple columns, making it harder to filter, group, or analyze.

To tidy the dataset, we need to reshape it using the `melt()` function.


### 3. Let's, again, use the `melt` function to fix the general structure of the dataframe.

In [5]:
# Reshape the dataset using melt without modifying column names
billboard_tidy = pd.melt(
    billboard_data,
    id_vars=[
        'year',
        'artist.inverted',
        'track',
        'time',
        'genre',
        'date.entered',
        'date.peaked'
    ],  # Keep metadata columns fixed
    var_name='week',              # All other columns become values in the 'week' column
    value_name='rank'             # Weekly chart positions
)

# Display the first few rows
billboard_tidy.head()


,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


The dataset has weekly rankings spread across multiple columns (e.g., `x1st.week`, `x2nd.week`, etc.). 
We use the `melt()` function to turn these columns into rows under a new `week` column, with the corresponding values stored in a `rank` column.


> If we inspect the current dataframe. We find that it is structured in a better way than before. 

> However, the ***Week*** column looks a bit ugly!

### 4. Let's try to place only the week number in that column without the extras surronding it.

In [8]:


# Step 2: Identify columns that represent weekly rankings
week_columns = [col for col in billboard_data.columns if col.startswith('x') and col.endswith('.week')]

# Step 3: Melt the week columns into a long format
billboard_tidy = pd.melt(
    billboard_data,
    id_vars=['year', 'artist.inverted', 'track', 'time', 'genre', 'date.entered', 'date.peaked'],
    value_vars=week_columns,
    var_name='week',
    value_name='rank'
)

# Step 4: Extract numeric week number from the 'week' column (e.g., from 'x1st.week' → 1)
billboard_tidy['week'] = billboard_tidy['week'].str.extract(r'(\d+)', expand=False)
billboard_tidy['week'] = pd.to_numeric(billboard_tidy['week'], errors='coerce')

# Step 5: Preview unique week numbers
billboard_tidy[['week']].drop_duplicates().sort_values('week').reset_index(drop=True).head(10)
# Display a preview of the cleaned and reshaped dataframe
billboard_tidy[['year', 'artist.inverted', 'track', 'time', 'genre',
                'date.entered', 'date.peaked', 'week', 'rank']].head(1000)



,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0
...,...,...,...,...,...,...,...,...,...
995,2000,"Badu, Erkyah",Bag Lady,5:03,Rock,2000-08-19,2000-10-07,4,41.0
996,2000,Pink,There U Go,3:23,Rock,2000-03-04,2000-04-08,4,11.0
997,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,4,63.0
998,2000,Nelly,(Hot S**t) Country Grammar,4:17,Rap,2000-04-29,2000-09-16,4,76.0



The original `week` column contains verbose labels such as `x1st.week`, `x2nd.week`, etc.
we extract just the numeric portion representing the actual week number.


### 5. Now let's inspect the ***Week*** column in the dataframe.

In [16]:
from IPython.display import display

unique_weeks = billboard_tidy['week'].dropna().drop_duplicates().sort_values().reset_index(drop=True)

unique_weeks_df = unique_weeks.to_frame(name='week_number')

display(unique_weeks_df.head(10).style.hide(axis='index'))


week_number
1
2
3
4
5
6
7
8
9
10



After reshaping the dataset, the `week` column contains values such as `x1st.week`, `x2nd.week`, etc. These are not in a usable numeric format.

We inspect these values to understand the pattern and prepare them for cleaning using regular expressions.


> Next, let's try to find the date at which the song ranked the number that is shown per row.

### 6. To do that let's first think of the equation that is going to get us the relevant date at which the song ranked the *rth*.



In [29]:
# Ensure 'entry_date' is in datetime format
billboard_tidy['entry_date'] = pd.to_datetime(billboard_tidy['entry_date'], errors='coerce')

# Compute the date the song charted during the r-th week
billboard_tidy['chart.date'] = billboard_tidy['entry_date'] + pd.to_timedelta((billboard_tidy['week'] - 1) * 7, unit='d')

# Preview the result with selected columns
from IPython.display import display

display(
    billboard_tidy[[
        'track', 'artist', 'entry_date', 'week', 'chart.date', 'rank'
    ]].dropna(subset=['chart.date']).head(10)
)


,track,artist,entry_date,week,chart.date,rank
0,Independent Women Part I,Destiny's Child,2000-09-23,1,2000-09-23,78.0
1,"Maria, Maria",Santana,2000-02-12,1,2000-02-12,15.0
2,I Knew I Loved You,Savage Garden,1999-10-23,1,1999-10-23,71.0
3,Music,Madonna,2000-08-12,1,2000-08-12,41.0
4,Come On Over Baby (All I Want Is You),"Aguilera, Christina",2000-08-05,1,2000-08-05,57.0
5,Doesn't Really Matter,Janet,2000-06-17,1,2000-06-17,59.0
6,Say My Name,Destiny's Child,1999-12-25,1,1999-12-25,83.0
7,Be With You,"Iglesias, Enrique",2000-04-01,1,2000-04-01,63.0
8,Incomplete,Sisqo,2000-06-24,1,2000-06-24,77.0
9,Amazed,Lonestar,1999-06-05,1,1999-06-05,81.0


*Timedeltas are absolute differences in times, expressed in difference units (e.g. days, hours, minutes, seconds). This method converts an argument from a recognized timedelta format / value into a Timedelta type.*


**Timedeltas** represent time differences.  
We use them to add week offsets to dates using `pd.to_timedelta()`.


***What is the problem with the calculation above?***

If `week` has missing values or `entry_date` is not in datetime format,  
the chart date calculation won't work correctly.  
We need to clean these columns first.

### 7. Let's only keep necessary columns

In [30]:
# Keep only relevant columns
billboard_tidy = billboard_tidy[[
    'year', 'artist', 'track', 'genre', 'entry_date', 'week', 'rank', 'date_charted'
]]

# Preview the cleaned DataFrame
billboard_tidy.head()


,year,artist,track,genre,entry_date,week,rank,date_charted
0,2000,Destiny's Child,Independent Women Part I,Rock,2000-09-23,1,78.0,2000-09-23
1,2000,Santana,"Maria, Maria",Rock,2000-02-12,1,15.0,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,Rock,1999-10-23,1,71.0,1999-10-23
3,2000,Madonna,Music,Rock,2000-08-12,1,41.0,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),Rock,2000-08-05,1,57.0,2000-08-05


### 8. How to rename your columns?

In [31]:
# Rename columns for clarity and consistency
billboard_tidy = billboard_tidy.rename(columns={
    'artist': 'artist_name',
    'track': 'song_title',
    'genre': 'music_genre',
    'entry_date': 'entry_date',
    'week': 'week_number',
    'rank': 'chart_rank',
    'date_charted': 'chart_date'
})

# Display the updated column names
billboard_tidy.head()


,year,artist_name,song_title,music_genre,entry_date,week_number,chart_rank,chart_date
0,2000,Destiny's Child,Independent Women Part I,Rock,2000-09-23,1,78.0,2000-09-23
1,2000,Santana,"Maria, Maria",Rock,2000-02-12,1,15.0,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,Rock,1999-10-23,1,71.0,1999-10-23
3,2000,Madonna,Music,Rock,2000-08-12,1,41.0,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),Rock,2000-08-05,1,57.0,2000-08-05


In the above dataframe, there are some *NaN* values. What are we going to do? <br/>

Some rows have `NaN` in the `chart_rank` column.  
This means the song was not on the chart that week.

We will remove these rows to keep only valid chart entries.

### 9. Apply quick data cleaning and then observe the dataset

## **Data Cleaning**

Data cleaning involves removing unwanted characters, imputing, or dropping missing values.

The decision is based on the dataset you have, and the information you can extract from the other columns.


Examples of data cleaning include cleaning:

1.   **Missing Data**
2.   **Irregular Data** (Outliers)
3.   **Unnecessary Data** — Repetitive Data, Duplicates and more
4.   **Inconsistent Data** — Capitalization, Addresses and mored


In [ ]:
# Check for missing values
missing_values = billboard_tidy.isna().sum()
# Detect duplicates
duplicate_count = billboard_tidy.duplicated().sum()
# Display inspection results
print("Missing Values:\n", missing_values)
print("\nDuplicate Rows:", duplicate_count)




Before cleaning, we check for:

- **Missing values** in each column.
- **Duplicate rows** in the dataset.

This helps us understand data quality and decide how to handle inconsistencies.


In [ ]:
# 1. Drop rows with missing critical values: 'chart_rank' or 'chart_date'
billboard_tidy = billboard_tidy.dropna(subset=['chart_rank', 'chart_date']).copy()

# 2. Remove exact duplicate rows
billboard_tidy = billboard_tidy.drop_duplicates()

# 3. Normalize column names: lowercase, stripped, underscores instead of spaces
billboard_tidy.columns = [col.strip().lower().replace(' ', '_') for col in billboard_tidy.columns]

# 4. Preview the cleaned DataFrame
billboard_tidy.head()




1. **Remove rows** where `chart_rank` or `chart_date` is missing.
2. **Drop duplicate rows** to avoid repetition.
3. **Standardize column names** by:
   - Converting to lowercase
   - Replacing spaces with underscores
   - Stripping any extra whitespace

Finally, we preview the cleaned DataFrame.

---


### **Cars Data Set**

Start by reading the dataset related to car models: ./CSVs/cars.csv

First Row seems to be the datatype, we need to remove it

Let's observe the columns with null values.  Either by using the `isnull().sum()` function 

There aren't many missing values. Let's take a glimpse at the percentage of the missing values:

**HINT:** We'll need `Numpy` for the below task.

Around *0.19%* of the values are missing, which isn't a lot. Therefore, we might go with the option of dropping all the rows with null values.

Lets also check dropping the columns

Let's observe how many columns we lost

### **Cars Dataset - Filling in missing values automatically**

Another option is to try and fill in the missing values through imputations.

Let's take the `MPG` column for example. We can fill in the missing values with 0s through the following line of code:

`df_cars.fillna(0) `. 

However, this does not make much sense as there isn't MPG equal to 0. How about we plot the MPG column and if it follows a random distribution we can use the mean of the column to compute the missing values. Otherwise, we can use the median (if there is a skewed normal distribution). However, there might be a better way of imputation which is getting the median or the mean of the MPG of the cars with similar attributes.

If we observe the graph above, we can consider it in a way or another normally distributed. Therefore, we can impute the missing values using the mean.

To compute the mean we need numeric values. However the values in the dataframe are objects. Therefore, we need to change them to numerics so that we can compute them.

Now let's see what is the mean of the MPG column

We can use this mean to compute the missing values since the graph demonstarted a normal distribution

### **Car Dataset - Simple Imputer**

*SimpleImputer* is a `scikit-learn` class which is helpful in handling the missing data in the predictive model dataset. It replaces the `NaN` values with a specified placeholder.
It is implemented by the use of the `SimpleImputer()` method which takes the following arguments :

`missing_values` : The missing_values placeholder which has to be imputed. By default is NaN

`strategy` : The data which will replace the NaN values from the dataset. The strategy argument can take the values – ‘mean'(default), ‘median’, ‘most_frequent’ and ‘constant’.


Let's start by importing the `SimpleImputer` into our notebook

What we need to do are two essential steps:

1. fit the data (compute the mean / median / most freq)
2. transform the data (place the computed values in the NaN cells)

## **Outlier Detection** 


An Outlier is a data-item/object that deviates significantly from the rest of the (so-called normal)objects. They can be caused by measurement or execution errors. The analysis for outlier detection is referred to as outlier mining. There are many ways to detect the outliers, and the removal process is the data frame same as removing a data item from the panda’s data frame.



https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/





### Outliers Visualization

#### Visualizing Outliers Using Box Plot
It captures the summary of the data effectively and efficiently with only a simple box and whiskers. Boxplot summarizes sample data using 25th, 50th, and 75th percentiles. One can just get insights(quartiles, median, and outliers) into the dataset by just looking at its boxplot.

#### Visualizing Outliers Using ScatterPlot.

It is used when you have paired numerical data and when your dependent variable has multiple values for each reading independent variable, or when trying to determine the relationship between the two variables. In the process of utilizing the scatter plot, one can also use it for outlier detection.

#### Z-Score:
Z- Score is also called a standard score. This value/score helps to understand that how far is the data point from the mean. And after setting up a threshold value one can utilize z score values of data points to define the outliers.
<br>
Zscore = (data_point -mean) / std. deviation



Now to define an outlier threshold value is chosen which is generally 3.0. As 99.7% of the data points lie between +/- 3 standard deviation (using Gaussian Distribution approach).

Rows where Z value is greater than 2

#### IQR (Inter-Quartile Range)
Inter Quartile Range approach to finding the outliers is the most commonly used and most trusted approach used in the research field. <Br>
IQR = Quartile3 - Quartile1

To define the outlier base value is defined above and below dataset’s normal range namely Upper and Lower bounds, define the upper and the lower bound (1.5*IQR value is considered) :<br>
upper = Q3 + 1.5 * IQR <br>
lower = Q1 - 1.5 * IQR <br> 

- Removing the outliers:
For removing the outlier, one must follow the same process of removing an entry from the dataset using its exact position in the dataset because in all the above methods of detecting the outliers end result is the list of all those data items that satisfy the outlier definition according to the method used.